In [3]:
###load libraries and set parameters
import os
import sys
import matplotlib.pyplot as plt
import sklearn
import librosa.display
import numpy
import json

# Check for duplicates
First of all check if any of the selected song has the same name, and if so, to avoid problems modify the names.

In [ ]:
###TODO: write a function to check for duplicate

# Parse song to extract features
Here we provide the code to extract the mffc from the song in our database. We extract both normalized *song-by-song* and unnormalized mfcc's. Normalized ones are usefull for the classification while for visualization purposes is better to look at unormalized features and eventually normalize them globally (not *song by song*).

In [4]:
###set parameters:

fs=6000      #sampling rate, default in librosa is 22050
offset=30    #load song with specified offset in sec
duration=60  #portion of the song to load in sec
n_mfcc=12    #number of mfccs to extract, default is 20, other good value (to reduce dimension) is 12

In [6]:
###define function to extract features using parameter set above
scaler = sklearn.preprocessing.StandardScaler()##to normalize mfcc
def create_mfcc(walk_dir):
    x = []
    x1 = []
    #c = 0
    for root, subdirs, files in os.walk(walk_dir):
        for filename in files:
            if ".mp3" in filename:
                file_path = os.path.join(root, filename)
                artist=root.split("\\")[1]
                song=filename.split(" ",1)[1].split(".")[0]
                print("processing:"+song)
                a = librosa.load(file_path,duration=duration,offset=offset,sr=fs)[0]
                mfcc = librosa.feature.mfcc(a, sr=fs,n_mfcc=n_mfcc).T
                mfcc_s = scaler.fit_transform(mfcc)
                x.append({"artist": artist,"song": song,"mfcc":mfcc_s})
                x1.append({"artist": artist,"song": song,"mfcc":mfcc})
    return(x,x1)

'###define function to extract features using parameter set above\nscaler = sklearn.preprocessing.StandardScaler()##to normalize mfcc\ndef create_mfcc(walk_dir):\n    x = []\n    x1 = []\n    #c = 0\n    for root, subdirs, files in os.walk(walk_dir):\n        for filename in files:\n            if ".mp3" in filename:\n                file_path = os.path.join(root, filename)\n                artist=root.split("\\")[1]\n                song=filename.split(" ",1)[1].split(".")[0]\n                print("processing:"+song)\n                a = librosa.load(file_path,duration=duration,offset=offset,sr=fs)[0]\n                mfcc = librosa.feature.mfcc(a, sr=fs,n_mfcc=n_mfcc).T\n                mfcc_s = scaler.fit_transform(mfcc)\n                x.append({"artist": artist,"song": song,"mfcc":mfcc_s})\n                x1.append({"artist": artist,"song": song,"mfcc":mfcc})\n    return(x,x1)'

In [ ]:
#create a python list of dictionaries {artist, song, mfcc} using above function
#x1 are non-nomalized and will be useful for visualization purposes
x,x1=create_mfcc("canzoni")

In [5]:
#save files with extracted features

##convert mfcc from numpy array to list to be able to place them in a json file
y=[]
for t in x:
    y.append({"artist": t["artist"],"song": t["song"],"mfcc":t["mfcc"].tolist()})   
y1=[]
for t in x1:
    y1.append({"artist": t["artist"],"song": t["song"],"mfcc":t["mfcc"].tolist()})

#save features
with open('x.txt', 'w') as filehandle:
    json.dump(y, filehandle)
with open('x1.txt', 'w') as filehandle:
    json.dump(y1, filehandle)
    
#save also the sampling rate used to extract features
with open('fs.txt', 'w') as filehandle:
    json.dump(fs, filehandle)